# Applying many similar aggregations.

Now suppose we want to compute a large number of similar aggregations, one for each of a collection of columns.  A similar approach will allow us to accomplish this task regardless of the number of columns.

In [2]:
import polars as pl

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Artwork

**Note.** The last two columns should also be of type `pl.Float`.  Let's define a schema using a `dict` comprehension.

In [3]:
artwork = (pl.read_csv("./data/Artworks.csv", infer_schema_length=50000)
          )
artwork.head(2)

Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str,str
"""Ferdinandsbrücke Project, Vien…","""Otto Wagner""","""6210""","""(Austrian, 1841–1918)""","""(Austrian)""","""(1841)""","""(1918)""","""(Male)""","""1896""","""Ink and cut-and-pasted painted…","""19 1/8 x 66 1/2"" (48.6 x 168.9…","""Fractional and promised gift o…","""885.1996""","""Architecture""","""Architecture & Design""","""1996-04-09""","""Y""",2,"""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,48.6,null,null,168.9,null,null
"""City of Music, National Superi…","""Christian de Portzamparc""","""7470""","""(French, born 1944)""","""(French)""","""(1944)""","""(0)""","""(Male)""","""1987""","""Paint and colored pencil on pr…","""16 x 11 3/4"" (40.6 x 29.8 cm)""","""Gift of the architect in honor…","""1.1995""","""Architecture""","""Architecture & Design""","""1995-01-17""","""Y""",3,"""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,40.6401,null,null,29.8451,null,null


###  Comprehensions EVERYWHERE!

**Comprehension mindset.** Whenever creating a `list`/`tuple`/`dict`/`set`, automate the process with a comprehension.

**Current task.** Create a schema `dict` that correctly sets all dimension columns to `float`.

**Strategy.** Use a comprehension and the fact all dimension columns end in `')'` to create the schema.

In [4]:
artwork_schema = {col: pl.Float64 if col.endswith(')') else pl.String for col in artwork.columns}

artwork_schema

{'Title': String,
 'Artist': String,
 'ConstituentID': String,
 'ArtistBio': String,
 'Nationality': String,
 'BeginDate': String,
 'EndDate': String,
 'Gender': String,
 'Date': String,
 'Medium': String,
 'Dimensions': String,
 'CreditLine': String,
 'AccessionNumber': String,
 'Classification': String,
 'Department': String,
 'DateAcquired': String,
 'Cataloged': String,
 'ObjectID': String,
 'URL': String,
 'ThumbnailURL': String,
 'Circumference (cm)': Float64,
 'Depth (cm)': Float64,
 'Diameter (cm)': Float64,
 'Height (cm)': Float64,
 'Length (cm)': Float64,
 'Weight (kg)': Float64,
 'Width (cm)': Float64,
 'Seat Height (cm)': Float64,
 'Duration (sec.)': Float64}

In [5]:
artwork = (pl.read_csv("./data/Artworks.csv", schema=artwork_schema)
          )
artwork.head(2)

Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Ferdinandsbrücke Project, Vien…","""Otto Wagner""","""6210""","""(Austrian, 1841–1918)""","""(Austrian)""","""(1841)""","""(1918)""","""(Male)""","""1896""","""Ink and cut-and-pasted painted…","""19 1/8 x 66 1/2"" (48.6 x 168.9…","""Fractional and promised gift o…","""885.1996""","""Architecture""","""Architecture & Design""","""1996-04-09""","""Y""","""2""","""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,48.6,null,null,168.9,null,null
"""City of Music, National Superi…","""Christian de Portzamparc""","""7470""","""(French, born 1944)""","""(French)""","""(1944)""","""(0)""","""(Male)""","""1987""","""Paint and colored pencil on pr…","""16 x 11 3/4"" (40.6 x 29.8 cm)""","""Gift of the architect in honor…","""1.1995""","""Architecture""","""Architecture & Design""","""1995-01-17""","""Y""","""3""","""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,40.6401,null,null,29.8451,null,null


## A WET transformation - using alias

This solution is WET, not DRY, as all 9 of the expressions in mutate are very similar ... if only here was a way to perform these similar expressions all at once ...

In [6]:
# Input expressions as positional arguments.
(artwork
 .group_by('Classification')
 .agg(pl.col('Circumference (cm)').mean().alias('Mean Circumference (cm)'),
      pl.col('Depth (cm)').mean().alias('Mean Depth (cm)'),
      pl.col('Diameter (cm)').mean().alias('Mean Diameter (cm)'),
      pl.col('Height (cm)').mean().alias('Mean Height (cm)'),
      pl.col('Length (cm)').mean().alias('Mean Length (cm)'),
      pl.col('Weight (kg)').mean().alias('Mean Weight (kg)'),
      pl.col('Width (cm)').mean().alias('Mean Width (cm)'),
      pl.col('Seat Height (cm)').mean().alias('Mean Seat Height (cm)'),
      pl.col('Duration (sec.)').mean().alias('Mean Duration (sec.)'),
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Work on Paper""",NaN,1.209352,NaN,83.666016,NaN,NaN,78.392955,NaN,NaN
"""Sculpture""",NaN,57.139591,106.085079,105.494561,465.3702,1387.632147,103.907527,NaN,1860.0
"""Mies van der Rohe Archive""",NaN,22.5,NaN,41.44568,NaN,90.7194,56.195844,NaN,NaN
"""Graphic Design""",NaN,0.0,NaN,326.597574,NaN,NaN,54.28382,NaN,NaN
"""Design""",77.860067,25.939486,18.215681,53.600846,86.081085,523.133433,50.063999,NaN,6593.166667
…,…,…,…,…,…,…,…,…,…
"""Collage""",NaN,2.1875,NaN,70.355261,NaN,NaN,103.505515,NaN,NaN
"""Performance""",NaN,29.111299,NaN,130.39205,NaN,NaN,131.951426,NaN,600.0
"""Photography Research/Reference""",NaN,NaN,NaN,45.55,NaN,NaN,34.8,NaN,6120.0


In [7]:
# Input a list of expressions
(artwork
 .group_by('Classification')
 .agg([pl.col('Circumference (cm)').mean().alias('Mean Circumference (cm)'),
       pl.col('Depth (cm)').mean().alias('Mean Depth (cm)'),
       pl.col('Diameter (cm)').mean().alias('Mean Diameter (cm)'),
       pl.col('Height (cm)').mean().alias('Mean Height (cm)'),
       pl.col('Length (cm)').mean().alias('Mean Length (cm)'),
       pl.col('Weight (kg)').mean().alias('Mean Weight (kg)'),
       pl.col('Width (cm)').mean().alias('Mean Width (cm)'),
       pl.col('Seat Height (cm)').mean().alias('Mean Seat Height (cm)'),
       pl.col('Duration (sec.)').mean().alias('Mean Duration (sec.)'),
      ],
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Illustrated Book""",NaN,4.85699,14.471429,22.248024,58.858049,NaN,20.503477,NaN,NaN
"""Architecture""",NaN,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,NaN,2160.461538
"""Installation""",NaN,70.281632,136.220202,111.509097,582.520444,317.517917,116.929314,NaN,4917.898089
"""Software""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Graphic Design""",NaN,0.0,NaN,326.597574,NaN,NaN,54.28382,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Collage""",NaN,2.1875,NaN,70.355261,NaN,NaN,103.505515,NaN,NaN
"""Photography Research/Reference""",NaN,NaN,NaN,45.55,NaN,NaN,34.8,NaN,6120.0
"""Video""",NaN,0.266216,NaN,2.712987,NaN,NaN,2.716711,NaN,3029.279015


In [8]:
help(artwork.group_by().agg)

Help on method agg in module polars.dataframe.group_by:

agg(*aggs: 'IntoExpr | Iterable[IntoExpr]', **named_aggs: 'IntoExpr') -> 'DataFrame' method of polars.dataframe.group_by.GroupBy instance
    Compute aggregations for each group of a group by operation.

    Parameters
    ----------
    *aggs
        Aggregations to compute for each group of the group by operation,
        specified as positional arguments.
        Accepts expression input. Strings are parsed as column names.
    **named_aggs
        Additional aggregations, specified as keyword arguments.
        The resulting columns will be renamed to the keyword used.

    Examples
    --------
    Compute the aggregation of the columns for each group.

    >>> df = pl.DataFrame(
    ...     {
    ...         "a": ["a", "b", "a", "b", "c"],
    ...         "b": [1, 2, 1, 3, 3],
    ...         "c": [5, 4, 3, 2, 1],
    ...     }
    ... )
    >>> df.group_by("a").agg(pl.col("b"), pl.col("c"))  # doctest: +IGNORE_RESULT
    s

In [9]:
compute_mean = lambda col: pl.col(col).mean().alias(f'Mean {col}')

In [10]:
# Input a list of expressions
(artwork
 .group_by('Classification')
 .agg([compute_mean(c) for c in artwork.columns if c.endswith(')')]
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Textile""",NaN,8.724003,NaN,108.386216,103.0,NaN,118.780417,NaN,NaN
"""Mies van der Rohe Archive""",NaN,22.5,NaN,41.44568,NaN,90.7194,56.195844,NaN,NaN
"""Software""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Multiple""",29.516667,7.796307,9.303058,19.382265,26.65,2.783105,20.988888,NaN,60.0
"""Performance""",NaN,29.111299,NaN,130.39205,NaN,NaN,131.951426,NaN,600.0
…,…,…,…,…,…,…,…,…,…
"""Ephemera""",NaN,1.322905,4.75,27.317681,25.08255,NaN,24.492504,NaN,NaN
"""Installation""",NaN,70.281632,136.220202,111.509097,582.520444,317.517917,116.929314,NaN,4917.898089
"""Architecture""",NaN,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,NaN,2160.461538


In [11]:
# Unpacking positional arguments
(artwork
 .group_by('Classification')
 .agg(*[compute_mean(c) for c in artwork.columns if c.endswith(')')]
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Frank Lloyd Wright Archive""",NaN,3.738871,26.670053,55.780718,NaN,NaN,70.306716,NaN,NaN
"""Painting""",NaN,8.087142,138.357934,122.422894,NaN,96.285524,130.379904,NaN,NaN
"""Performance""",NaN,29.111299,NaN,130.39205,NaN,NaN,131.951426,NaN,600.0
"""Video""",NaN,0.266216,NaN,2.712987,NaN,NaN,2.716711,NaN,3029.279015
"""Poster""",NaN,0.0,NaN,98.16542,NaN,NaN,70.427396,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Design""",77.860067,25.939486,18.215681,53.600846,86.081085,523.133433,50.063999,NaN,6593.166667
"""Photography Research/Reference""",NaN,NaN,NaN,45.55,NaN,NaN,34.8,NaN,6120.0
"""Film (object)""",NaN,NaN,NaN,8.7,NaN,NaN,9.4,NaN,NaN


In [12]:
dimension_col_means = lambda df: [compute_mean(c) for c in df.columns if c.endswith(')')]

In [13]:
# List of input expressions
(artwork
 .group_by('Classification')
 .agg(dimension_col_means(artwork)
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Performance""",NaN,29.111299,NaN,130.39205,NaN,NaN,131.951426,NaN,600.0
"""Installation""",NaN,70.281632,136.220202,111.509097,582.520444,317.517917,116.929314,NaN,4917.898089
"""Software""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Photograph""",NaN,3.372236,8.041579,28.519292,NaN,226.798512,31.585365,NaN,280.0
"""Poster""",NaN,0.0,NaN,98.16542,NaN,NaN,70.427396,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Architecture""",NaN,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,NaN,2160.461538
"""Digital""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4590.0
"""Media""",NaN,1.696429,NaN,12.47931,NaN,NaN,13.668966,NaN,NaN


In [14]:
# Unpacking positional arguments
(artwork
 .group_by('Classification')
 .agg(*dimension_col_means(artwork)
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Print""",38.0,1.716388,49.066169,40.158467,14.5,1.5,37.848143,null,0.0
"""Audio""",null,0.17894,19.685039,11.84564,null,null,11.942762,null,1327.666667
"""Software""",null,null,null,null,null,null,null,null,null
"""Design""",77.860067,25.939486,18.215681,53.600846,86.081085,523.133433,50.063999,null,6593.166667
"""Sculpture""",null,57.139591,106.085079,105.494561,465.3702,1387.632147,103.907527,null,1860.0
…,…,…,…,…,…,…,…,…,…
"""Frank Lloyd Wright Archive""",null,3.738871,26.670053,55.780718,null,null,70.306716,null,null
"""Film (object)""",null,null,null,8.7,null,null,9.4,null,null
"""Ephemera""",null,1.322905,4.75,27.317681,25.08255,null,24.492504,null,null


## <font color="red"> Exercise 3.6</font>

Now suppose that we want to compute both the mean and standard deviation of each dimension.  Perform this task using an approach to the one shown above, this the results as DRY as possible. **Hint.** You can add two lists of transformations.

In [18]:
compute_mean = lambda col: pl.col(col).mean().alias(f'Mean {col}')

In [30]:
compute_std = lambda col: pl.col(col).std().alias(f'Std {col}')

In [42]:
dimension_col_means = lambda df: [compute_mean(c) for c in df.columns if c.endswith(')')]

In [51]:
dimension_col_std = lambda df: [compute_std(c) for c in df.columns if c.endswith(')')]

In [79]:
(artwork
 .group_by('Classification')
 .agg(
      dimension_col_std(artwork)
     )
)

Classification,Std Circumference (cm),Std Depth (cm),Std Diameter (cm),Std Height (cm),Std Length (cm),Std Weight (kg),Std Width (cm),Std Seat Height (cm),Std Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Drawing""",null,5.541451,113.141927,37.563622,null,21.168939,126.541794,null,null
"""Work on Paper""",null,9.317436,null,56.24971,null,null,250.423161,null,null
"""Digital""",null,null,null,null,null,null,null,null,7364.007061
"""Sculpture""",null,96.724075,178.014519,248.935986,337.211086,12544.483721,151.869626,null,null
"""Ephemera""",null,8.041355,1.06066,17.979945,4.041123,null,25.435286,null,null
…,…,…,…,…,…,…,…,…,…
"""Illustrated Book""",null,13.199276,9.790423,12.668747,11.32001,null,14.614537,null,null
"""Mies van der Rohe Archive""",null,59.529404,null,25.204271,null,null,38.151564,null,null
"""Collage""",null,1.402485,null,18.081858,null,null,24.304539,null,null


In [72]:
dimension_col_means_std = lambda df: [(compute_mean(c), compute_std(c)) for c in df.columns if c.endswith(')')]

In [78]:
# List of input expressions
(artwork
 .group_by('Classification')
 .agg(
      dimension_col_means_std(artwork)
     )
)

TypeError: not yet implemented: Nested object types

Hint: Try setting `strict=False` to allow passing data with mixed types.